In [7]:
# imports
import requests
import os
import json
import pandas as pd

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [8]:
key = os.environ["FOURSQUARE_KEY"]
headers = {
    "accept": "application/json",
    "Authorization": key
}

foursquare_poi = {}

for station in stations[:250]: # Using 250 stations for processing time
    url = "https://api.foursquare.com/v3/places/search?ll=" + str(station["latitude"]) + "%2C" + str(station["longitude"]) + "&radius=1000&fields=name,categories,location,distance,rating&sort=RELEVANCE"
    response = json.loads(requests.get(url, headers=headers).text)
    foursquare_poi[station["station_name"]] = response

print(foursquare_poi)

{'001023 - River Street , Clerkenwell': {'results': [{'categories': [{'id': 16032, 'name': 'Park', 'short_name': 'Park', 'plural_name': 'Parks', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}}], 'distance': 150, 'location': {'address': '42 Myddelton Square', 'admin_region': 'England', 'country': 'GB', 'cross_street': '', 'formatted_address': '42 Myddelton Square, London, Greater London, EC1R 1YB', 'locality': 'London', 'post_town': 'London', 'postcode': 'EC1R 1YB', 'region': 'Greater London'}, 'name': 'Myddelton Square', 'rating': 8.1}, {'categories': [{'id': 17000, 'name': 'Retail', 'short_name': 'Retail', 'plural_name': 'Retail', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_', 'suffix': '.png'}}], 'distance': 280, 'location': {'address': '23 Arlington Way', 'admin_region': 'England', 'country': 'GB', 'formatted_address': '23 Arlington Way, London, Greater London, EC1R 1UY', 'locality': 'London', 'post_to

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [9]:
clean_foursquare_poi = []

for station in foursquare_poi:
    for poi in foursquare_poi[station]["results"]:
        clean_foursquare_poi.append(
            {
                "station_name": station,
                "station_distance_(m)": poi["distance"],
                "loc_address": poi["location"]["formatted_address"],
                "loc_name": poi["name"],
                "loc_category": poi["categories"][0]["name"],
                "loc_rating": poi.get("rating", None)
            }
        )

Put your parsed results into a DataFrame

In [10]:
foursquare_df = pd.DataFrame(clean_foursquare_poi)
foursquare_df

,station_name,station_distance_(m),loc_address,loc_name,loc_category,loc_rating
0,"001023 - River Street , Clerkenwell",150,"42 Myddelton Square, London, Greater London, E...",Myddelton Square,Park,8.1
1,"001023 - River Street , Clerkenwell",280,"23 Arlington Way, London, Greater London, EC1R...",Present & Correct,Retail,8.8
2,"001023 - River Street , Clerkenwell",294,"96-98 Pentonville Rd, London, Greater London, ...",The Lexington,Rock Club,8.4
3,"001023 - River Street , Clerkenwell",325,"399 St John St, London, Greater London, EC1V 4LD",Turner & George,Butcher,8.7
4,"001023 - River Street , Clerkenwell",330,"67 Exmouth Market (Tysoe St), London, Greater ...",Briki,Bakery,8.7
...,...,...,...,...,...,...
2495,"200191 - Spencer Park, Wandsworth Common",346,"123 St John's Hill (Vardens Rd), Wandsworth, G...",Birdhouse,Coffee Shop,8.2
2496,"200191 - Spencer Park, Wandsworth Common",374,"162 St John's Hill, London, Greater London, SW...",St Johns Therapy Centre,Medical Center,6.7
2497,"200191 - Spencer Park, Wandsworth Common",603,"69 Northcote Rd, London, Greater London, SW11 1NP",Oddono's Gelati,Bakery,8.6
2498,"200191 - Spencer Park, Wandsworth Common",602,"76 Northcote Rd, Battersea, Greater London, SW...",Franco Manca,Pizzeria,8.4


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [11]:
key = os.environ["YELP_KEY"]
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {key}"
}

yelp_poi = {}

for station in stations[:250]:
    url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(station["latitude"]) + "&longitude=" + str(station["longitude"]) + "&radius=1000&categories=&sort_by=best_match&limit=10" # Match Foursquare default limit of 10
    response = json.loads(requests.get(url, headers=headers).text)
    yelp_poi[station["station_name"]] = response

print(yelp_poi)

{'001023 - River Street , Clerkenwell': {'error': {'code': 'ACCESS_LIMIT_REACHED', 'description': "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://docs.developer.yelp.com/docs/fusion-rate-limiting"}}, '001018 - Phillimore Gardens, Kensington': {'error': {'code': 'ACCESS_LIMIT_REACHED', 'description': "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://docs.developer.yelp.com/docs/fusion-rate-limiting"}}, '001024 - Park Street, Bankside': {'error': {'code': 'ACCESS_LIMIT_REACHED', 'description': "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://docs.developer.yelp.com/docs/fusion-rate-limiting"}}, '001022 - Brunswick Square, Bloomsbury': {'error': {'code': 'ACCESS_LIMIT_REACHED', 'description': "You've reached the access limit for this client. See instructions for requesting a higher access limit

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [12]:
clean_yelp_poi = []

for station in yelp_poi:
    for poi in yelp_poi[station]["businesses"]:
        clean_yelp_poi.append(
            {
                "station_name": station,
                "station_distance_(m)": round(poi["distance"]),
                "loc_address": poi["location"]["address1"],
                "loc_name": poi["name"],
                "loc_category": poi["categories"][0]["title"],
                "loc_rating": poi["rating"]
            }
        )

KeyError: 'businesses'

Put your parsed results into a DataFrame

In [ ]:
yelp_df = pd.DataFrame(clean_yelp_poi)
yelp_df

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

'''

Both APIs actually provided enough data for the information relevant to what I am comparing. By using the API parameters and field filtering, I can find the name, location, rating, etc. for all POIs. However, the rating scale provided by the APIs is different; Foursquare provides ratings on a scale /10, whereas Yelp uses a rating scale /5. This will make it difficult to compare the ratings of POIs that are sourced from different APIs.

'''

Get the top 10 restaurants according to their rating

In [ ]:
foursquare_restaurants = foursquare_df.loc[foursquare_df["loc_category"] == "Restaurant"]
foursquare_restaurants = foursquare_restaurants.sort_values(by="loc_rating", ascending=False)
foursquare_restaurants.head(10)

In [ ]:
restaurants = ["Restaurant", "Chinese", "Lebanese", "Vietnamese", "Italian", "British", "Pizza", "Turkish", "Sandwiches"] # The Yelp API doesn't categorize restaurants as a general term

yelp_restaurants = yelp_df.loc[yelp_df["loc_category"].isin(restaurants)]
yelp_restaurants = yelp_restaurants.sort_values(by="loc_rating", ascending=False)
yelp_restaurants.head(10)